In [24]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 4000
import datetime as dt
import re

In [46]:
tst = pd.read_csv('akram_files/timing_test_events1.txt', delimiter = '\t',\
                   header = None, names = ['pin_num', 'label', 'date', 'time'])
tst.head()

,pin_num,label,date,time
0,BioLab,Acquisition Started,03/06/2023,03:35:06.294 PM
1,sync summary event # 13,start_experiment,03/06/2023,03:35:17.438 PM
2,sync summary event # 128,async summary event 128,03/06/2023,03:35:17.469 PM
3,sync summary event # 2,describe_trial_start,03/06/2023,03:35:26.346 PM
4,sync summary event # 128,async summary event 128,03/06/2023,03:35:26.417 PM


In [47]:
tst['label'] = tst['label'].replace('async summary event 128', 'photo_trigger')
tst.head()

,pin_num,label,date,time
0,BioLab,Acquisition Started,03/06/2023,03:35:06.294 PM
1,sync summary event # 13,start_experiment,03/06/2023,03:35:17.438 PM
2,sync summary event # 128,photo_trigger,03/06/2023,03:35:17.469 PM
3,sync summary event # 2,describe_trial_start,03/06/2023,03:35:26.346 PM
4,sync summary event # 128,photo_trigger,03/06/2023,03:35:26.417 PM


In [38]:
tst2 = tst[(tst.label.str.contains('end', regex = True) == False)]
tst3 = tst2[tst2['label'] != 'fixation']
tst3.loc[tst['label'] != 'Acquisition Started']

,trigger,label,date,time
1,sync summary event # 13,start_experiment,03/06/2023,03:35:17.438 PM
2,sync summary event # 128,photo_trigger,03/06/2023,03:35:17.469 PM
3,sync summary event # 2,describe_trial_start,03/06/2023,03:35:26.346 PM
4,sync summary event # 128,photo_trigger,03/06/2023,03:35:26.417 PM
5,sync summary event # 6,describe_trial_choice_start,03/06/2023,03:35:36.385 PM
6,sync summary event # 128,photo_trigger,03/06/2023,03:35:36.433 PM
9,sync summary event # 1,imagine_trial_start,03/06/2023,03:35:42.031 PM
10,sync summary event # 128,photo_trigger,03/06/2023,03:35:42.083 PM
11,sync summary event # 5,imagine_trial_choice_start,03/06/2023,03:35:52.050 PM
12,sync summary event # 128,photo_trigger,03/06/2023,03:35:52.099 PM


In [70]:

tst.iloc[1]

pin_num    sync summary event # 13
label             start_experiment
date                    03/06/2023
time               03:35:17.438 PM
Name: 1, dtype: object

In [66]:
clean_triggers = pd.DataFrame({'pin_num': [], 'label': [], 'date': [], 'time': []})
for loc in range(0, len(tst['label']) - 1):
    if tst['label'][loc + 1] == 'photo_trigger':
        clean_triggers = pd.concat([clean_triggers, tst.iloc[loc], tst.iloc[loc + 1]])
clean_triggers

1
3
5
9
11
15
17
21
23
27
29
33
35
37
40
42
44
47
50
53
55
59
61
65
67
71
73
77
79
83
85
89
91
95
97
101
103
107
109
113
115
119
121
125
127
131
138
140
144
149
151
155
157
161
163
167
169
173
175
179
181
185
187
191
193
197
199
203
205
209
211
215
220
222
226
228
232
234
238
240
244
246
250
252
258
261
264
266
270
275
277
281
283
287
289
293
295
299
301
305
307
311
313
317
322
324
328
330
334
336
340
342
346
348
352
354
358
360
364
366
370
372
376
378
382
384
388
389
391
395
400
402
406
408


,pin_num,label,date,time,0
pin_num,NaN,NaN,NaN,NaN,sync summary event # 13
label,NaN,NaN,NaN,NaN,start_experiment
date,NaN,NaN,NaN,NaN,03/06/2023
time,NaN,NaN,NaN,NaN,03:35:17.438 PM
pin_num,NaN,NaN,NaN,NaN,sync summary event # 128
label,NaN,NaN,NaN,NaN,photo_trigger
date,NaN,NaN,NaN,NaN,03/06/2023
time,NaN,NaN,NaN,NaN,03:35:17.469 PM
pin_num,NaN,NaN,NaN,NaN,sync summary event # 2
label,NaN,NaN,NaN,NaN,describe_trial_start


In [49]:
tst

,pin_num,label,date,time
0,BioLab,Acquisition Started,03/06/2023,03:35:06.294 PM
1,sync summary event # 13,start_experiment,03/06/2023,03:35:17.438 PM
2,sync summary event # 128,photo_trigger,03/06/2023,03:35:17.469 PM
3,sync summary event # 2,describe_trial_start,03/06/2023,03:35:26.346 PM
4,sync summary event # 128,photo_trigger,03/06/2023,03:35:26.417 PM
5,sync summary event # 6,describe_trial_choice_start,03/06/2023,03:35:36.385 PM
6,sync summary event # 128,photo_trigger,03/06/2023,03:35:36.433 PM
7,sync summary event # 10,describe_trial_choice_end,03/06/2023,03:35:39.963 PM
8,sync summary event # 14,fixation,03/06/2023,03:35:40.999 PM
9,sync summary event # 1,imagine_trial_start,03/06/2023,03:35:42.031 PM


In [50]:
len(tst)

413

In [51]:
len(tst['label'])

413

In [52]:
for i in range(0,2):
    print(i)

0
1
